# Format and export the experimental results

#### Authors

* Juan Carlos Alfaro Jiménez
* Juan Ángel Aledo Sánchez
* José Antonio Gámez Martín

In this notebook, we format and export (`.csv` and `.tex` files) the experimental results. Below, we detail the steps.

## 1. Arguments

First, we add the command line arguments:

In [ ]:
import argparse

In [ ]:
description = "Format and export the experimental results."

In [ ]:
parser = argparse.ArgumentParser(description=description)

* The path to the experimental results:

In [ ]:
arg = "--source"

In [ ]:
type = str

In [ ]:
help = "Path to the experimental results"

In [ ]:
parser.add_argument(arg, type=type, help=help);

* The path to the tables:

In [ ]:
arg = "--destination"

In [ ]:
type = str

In [ ]:
help = "Path to the tables"

In [ ]:
parser.add_argument(arg, type=type, help=help);

* The name of the target output variable:

In [ ]:
arg = "--output"

In [ ]:
default = "test_score"

In [ ]:
type = str

In [ ]:
help = "Name of the target output variable"

In [ ]:
parser.add_argument(arg, default=default, type=type, help=help);

* The number of decimal digits for the numeric output:

In [ ]:
arg = "--digits"

In [ ]:
default = 3

In [ ]:
type = int

In [ ]:
help = "Number of decimal digits for the numeric output"

In [ ]:
parser.add_argument(arg, default=default, type=type, help=help);

* The methods to filter:

In [ ]:
arg = "--methods"

In [ ]:
default = ".*"

In [ ]:
type = str

In [ ]:
help = "Methods to filter"

In [ ]:
parser.add_argument(arg, default=default, type=type, help=help);

* The datasets to filter:

In [ ]:
arg = "--datasets"

In [ ]:
default = ".*"

In [ ]:
type = str

In [ ]:
help = "Datasets to filter"

In [ ]:
parser.add_argument(arg, default=default, type=type, help=help);

Now, we parse the command line arguments:

In [ ]:
from pathlib import Path

In [ ]:
arguments = Path("arguments.txt").read_text("utf-8").split(" ")

In [ ]:
args = parser.parse_args(arguments)

And rename the variables:

In [ ]:
source = args.source

In [ ]:
source

In [ ]:
destination = args.destination

In [ ]:
destination

In [ ]:
output = args.output

In [ ]:
output

In [ ]:
digits = args.digits

In [ ]:
digits

In [ ]:
methods = args.methods

In [ ]:
methods

In [ ]:
datasets = args.datasets

In [ ]:
datasets

## 2. Load

Second, we get the files with the experiments results (`source`):

In [ ]:
import os

In [ ]:
pattern = os.path.join("work", source, "**", "*.csv")

In [ ]:
import glob

In [ ]:
files = glob.glob(pattern, recursive=True)

Now, we filter the methods (`methods`) and datasets (`datasets`):

In [ ]:
import re

In [ ]:
pattern = os.path.join("work", ".*", datasets, methods + ".csv")

In [ ]:
r = re.compile(pattern)

In [ ]:
files = filter(r.match, files)

Finally, we read the experimental results for the target output variable (`output`) and include a column with the file:

In [ ]:
usecols = [output]

In [ ]:
import pandas as pd

In [ ]:
read_csv = lambda file: pd.read_csv(file, usecols=usecols).assign(file=file)

In [ ]:
objs = map(read_csv, files)

In [ ]:
results = pd.concat(objs)

In [ ]:
results

## 3. Format

Third, we extract the method and the dataset from the filename:

In [ ]:
df = results.file.str.split(os.sep, expand=True)

In [ ]:
df = df.iloc[:, -2:]

In [ ]:
df.columns = ["dataset", "method"]

In [ ]:
df["method"] = df.method.str.replace(".csv", "", regex=False)

In [ ]:
df

And include this information in the experimental results:

In [ ]:
results = results.drop("file", axis=1)

In [ ]:
objs = [results, df]

In [ ]:
results = pd.concat(objs, axis=1)

In [ ]:
results

Now, we group by `method` and `dataset` and compute the mean and standard deviation:

In [ ]:
by = ["method", "dataset"]

In [ ]:
func = ["mean", "std"]

In [ ]:
results = results.groupby(by).agg(func)

In [ ]:
results

And extract the statistics:

In [ ]:
level = -1

In [ ]:
mean = results[output]["mean"].unstack(level=level)

In [ ]:
mean

In [ ]:
std = results[output]["std"].unstack(level=level)

In [ ]:
std

Then, we round by the number of decimal digits for the numeric output (`digits`):

In [ ]:
func = ("{:" + "." + "{digits}" + "f}").format

In [ ]:
mean = mean.round(digits).applymap(func, digits=digits)

In [ ]:
mean

In [ ]:
std = std.round(digits).applymap(func, digits=digits)

In [ ]:
std

Finally, we create a tabular with the mean and standard deviation:

In [ ]:
tabular = mean + " $ " + "\pm" + " $ " + std

In [ ]:
tabular

## 4. Export

Fourth, we export the mean and tabular information to the directory with the tables (`destination`):

In [ ]:
destination = os.path.join("work", destination, output)

In [ ]:
Path(destination).mkdir(parents=True, exist_ok=True)

In [ ]:
csv = os.path.join(destination, "mean.csv")

In [ ]:
mean.to_csv(csv)

In [ ]:
tex = os.path.join(destination, "tabular.tex")

In [ ]:
tabular.to_latex(tex, escape=False)

Finally, we write the destination directory in a file for the `HTML` export:

In [ ]:
Path("destination.txt").write_text(destination, encoding="utf-8");